## 幾種思路

思路1：TF-IDF + 機器學習分類器
直接使用TF-IDF對文本提取特徵，並使用分類器進行分類。在分類器的選擇上，可以使用SVM、LR、或者XGBoost。

思路2：FastText
FastText是入門款的詞向量，利用Facebook提供的FastText工具，可以快速構建出分類器。

思路3：WordVec + 深度學習分類器
WordVec是進階款的詞向量，並通過構建深度學習分類完成分類。深度學習分類的網絡結構可以選擇TextCNN、TextRNN或者BiLSTM。

思路4：Bert詞向量
Bert是高配款的詞向量，具有強大的建模學習能力。

## 獲取6種模型的資料集

In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import re
path = '../data/0414/review_data(seg+pos+stopwords)_n+v+f+p.csv'
df = pd.read_csv(path)

### 檢查重複值、空值

In [2]:
#印出重複資料
print(df[df.duplicated()])

Empty DataFrame
Columns: [reviews, value, comfort, location, cleanliness, service, facilities, ws_pos_reviews, filtered, filtered_word]
Index: []


In [3]:
#移除重複值
#df = df.drop_duplicates()
#print(df.shape)

In [4]:
#印出空值資料
#df_train[df_train.isnull().T.any()]

### 切分為6個資料集

In [5]:
def split_df(df):
    df_value = df[['value','filtered_word']]
    df_value.rename(columns={'value': 'label'}, inplace=True)
    df_comfort = df[['comfort','filtered_word']]
    df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
    df_location = df[['location','filtered_word']]
    df_location.rename(columns={'location': 'label'}, inplace=True)
    df_cleanliness = df[['cleanliness','filtered_word']]
    df_cleanliness.rename(columns={'cleanliness': 'label'}, inplace=True)
    df_service = df[['service','filtered_word']]
    df_service.rename(columns={'service': 'label'}, inplace=True)
    df_facilities = df[['facilities','filtered_word']]
    df_facilities.rename(columns={'facilities': 'label'}, inplace=True)
    return df_value, df_comfort, df_location, df_cleanliness, df_service, df_facilities

In [6]:
#df_value_train, df_comfort_train, df_location_train, df_cleanliness_train, df_service_train ,df_facilities_train = split_df(df_train)
#df_value_test, df_comfort_test, df_location_test, df_cleanliness_test, df_service_test, df_facilities_test = split_df(df_test)
df_value, df_comfort, df_location, df_cleanliness, df_service ,df_facilities = split_df(df)

C:\Users\g3190\AppData\Local\Temp/ipykernel_11344/3099386144.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_value.rename(columns={'value': 'label'}, inplace=True)
C:\Users\g3190\AppData\Local\Temp/ipykernel_11344/3099386144.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
C:\Users\g3190\AppData\Local\Temp/ipykernel_11344/3099386144.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [8]:
df_comfort

,label,filtered_word
0,1.0,"價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),好(VH..."
1,0.0,"內部(Ncd),房間(Nc),乾淨(VH),場地(Na),團體(Na),使用(VC),覺得(..."
2,1.0,"房間(Nc),小(VH),美中不足(VH),乾(VH),濕(VH),分離(VHC)"
3,1.0,"房子(Na),設計(VC),棒(VH),房間(Nc),採光(Na),好(VH),大廳(Nc)..."
4,1.0,"Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc..."
...,...,...
1267,0.0,"港式(Na),飲茶(VA),餐廳(Nc),口味(Na),棒(VH),環境(Na),乾淨(VH..."
1268,0.0,"場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(..."
1269,0.0,"交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(..."
1270,1.0,"地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),老闆娘(Na),親切(VH)..."


### 清理資料(移除詞性標註的文字)

In [9]:
def remove_N_comma(sentence):
    # 把後面(N..)(V..)(F..)拿掉
    sentence = str(sentence)
    pattern = re.compile(r"\([N,V,F,P].*?\)") #移除詞性標示
    sentence = re.sub(pattern, '', sentence)
    pattern = re.compile(r",") #將逗號替換為空格
    sentence = re.sub(pattern, ' ', sentence)
    return sentence
pd.options.mode.chained_assignment = None  # 忽略警告

In [10]:
#訓練集
df_comfort['filtered_word'] = df_comfort.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)

In [11]:
print(df_comfort.shape)
df_comfort

(1272, 2)


,label,filtered_word
0,1.0,價格 合理 舒適 房間 老闆娘 人 好 做 早餐 旅客 重點 早餐 吃到飽
1,0.0,內部 房間 乾淨 場地 團體 使用 覺得 棒
2,1.0,房間 小 美中不足 乾 濕 分離
3,1.0,房子 設計 棒 房間 採光 好 大廳 挑高 氣派 房價 合理 台東 住 民宿
4,1.0,Cp值 高 乾淨 舒適 空間 大樓 下 免費 吐司 咖啡 老闆 回復 速度
...,...,...
1267,0.0,港式 飲茶 餐廳 口味 棒 環境 乾淨 機車 汽車 停車位 位於 高鐵 附近 適合 宴客
1268,0.0,場地 氣派 丁香魚 酥脆 服務 親切 蠟味 蘿蔔糕 份量 一些 好 牛肉粥 好吃
1269,0.0,交通 方便 地下室 停車場 良好 菜色 好 空間 設計好 說 一流 飯店
1270,1.0,地點 佳 離 逢甲 夜市 老闆娘 親切 服務 房間 舒適 浴室 乾淨


## 模型架構

### 套件引用

In [12]:
#import package
#轉向量用
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
from scipy.sparse import coo_matrix

from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_validate
import pickle #儲存模型用
from sklearn.model_selection import train_test_split
#類別採樣
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling
import imblearn.combine as combine
from imblearn.pipeline import make_pipeline as make_pipeline_imb


#模型
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
#from xgboost.sklearn import XGBClassifier

#模型效能表現
import sklearn.metrics as metrics

### 顯示訓練結果

In [13]:
def classification_report(y_test, pre):
    #混淆矩陣
    confusion = metrics.confusion_matrix(y_test, pre)
    TP = confusion[1,1]
    TN = confusion[0,0]
    FP = confusion[0,1]
    FN = confusion[1,0]
    print("TP:",TP)
    print("TN:",TN)
    print("FP:",FP)
    print("FN:",FN)
    #Accuracy
    accuracy = (TP+TN)/float(TP+TN+FN+FP)
    print("Accuracy：", accuracy)
    #Sensitivity(Recall)
    recall = TP/float(TP+FN)
    print("Recall：", recall)
    #Specificity
    specificity = TN/float(TN+FP)
    print("Specificity：", specificity)
    #Precision
    precision = TP/float(TP+FP)
    print("Precision：", precision)
    #f1-score
    f1_score = ((2*precision*recall)/(precision+recall))
    print("f1_score：", f1_score)
    #AUC
    print("AUC：", metrics.roc_auc_score(y_test, pre))

In [14]:
df_comfort

,label,filtered_word
0,1.0,價格 合理 舒適 房間 老闆娘 人 好 做 早餐 旅客 重點 早餐 吃到飽
1,0.0,內部 房間 乾淨 場地 團體 使用 覺得 棒
2,1.0,房間 小 美中不足 乾 濕 分離
3,1.0,房子 設計 棒 房間 採光 好 大廳 挑高 氣派 房價 合理 台東 住 民宿
4,1.0,Cp值 高 乾淨 舒適 空間 大樓 下 免費 吐司 咖啡 老闆 回復 速度
...,...,...
1267,0.0,港式 飲茶 餐廳 口味 棒 環境 乾淨 機車 汽車 停車位 位於 高鐵 附近 適合 宴客
1268,0.0,場地 氣派 丁香魚 酥脆 服務 親切 蠟味 蘿蔔糕 份量 一些 好 牛肉粥 好吃
1269,0.0,交通 方便 地下室 停車場 良好 菜色 好 空間 設計好 說 一流 飯店
1270,1.0,地點 佳 離 逢甲 夜市 老闆娘 親切 服務 房間 舒適 浴室 乾淨


### 切分訓練、測試數據

In [15]:
from sklearn.model_selection import train_test_split
def split_label(df,seed):
    X = df.filtered_word.tolist()
    y = df.label
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=1/3,random_state=seed)
    return X_train,X_test,y_train,y_test

In [16]:
X_train_1,X_test_1,y_train_1,y_test_1 = split_label(df_comfort,1)
X_train_2,X_test_2,y_train_2,y_test_2 = split_label(df_comfort,2)
X_train_3,X_test_3,y_train_3,y_test_3 = split_label(df_comfort,3)

In [17]:
s1 = pd.Series(y_train_1)
freq1 = s1.value_counts() 
print(freq1) 
s2 = pd.Series(y_train_2)
freq2 = s2.value_counts() 
print(freq2) 
s3 = pd.Series(y_train_3)
freq3 = s3.value_counts() 
print(freq3) 

0.0    444
1.0    404
Name: label, dtype: int64
0.0    445
1.0    403
Name: label, dtype: int64
0.0    431
1.0    417
Name: label, dtype: int64


### 模型設計

#### (1) baseline

In [59]:
def SVM_model(X_train,X_test,y_train,y_test):
    print("SVM baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [60]:
def LR_model(X_train,X_test,y_train,y_test):
    print("LR baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), LogisticRegression(random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [61]:
def RF_model(X_train,X_test,y_train,y_test):
    print("RF baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), RandomForestClassifier(max_depth=2, random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [62]:
def AdaBoost_model(X_train,X_test,y_train,y_test):
    print("AdaBoost_model")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [63]:
def SVM_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(sampling_strategy='minority'), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [64]:
def SVM_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [65]:
def SVM_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [66]:
def SVM_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [67]:
def AdaBoost_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(sampling_strategy='minority'), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [68]:
def AdaBoost_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [69]:
def AdaBoost_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [70]:
def AdaBoost_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

## 模型訓練&結果

### 資料集1

#### baseline

In [71]:
SVM_model(X_train_1,X_test_1,y_train_1,y_test_1)
LR_model(X_train_1,X_test_1,y_train_1,y_test_1)
RF_model(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model(X_train_1,X_test_1,y_train_1,y_test_1)

SVM baseline
TP: 159
TN: 176
FP: 33
FN: 56
Accuracy： 0.7900943396226415
Recall： 0.7395348837209302
Specificity： 0.8421052631578947
Precision： 0.828125
f1_score： 0.7813267813267813
AUC： 0.7908200734394124


LR baseline
TP: 162
TN: 175
FP: 34
FN: 53
Accuracy： 0.7948113207547169
Recall： 0.7534883720930232
Specificity： 0.8373205741626795
Precision： 0.826530612244898
f1_score： 0.7883211678832116
AUC： 0.7954044731278513


RF baseline
TP: 87
TN: 200
FP: 9
FN: 128
Accuracy： 0.6768867924528302
Recall： 0.4046511627906977
Specificity： 0.9569377990430622
Precision： 0.90625
f1_score： 0.5594855305466238
AUC： 0.6807944809168799


AdaBoost_model
TP: 163
TN: 171
FP: 38
FN: 52
Accuracy： 0.7877358490566038
Recall： 0.7581395348837209
Specificity： 0.8181818181818182
Precision： 0.8109452736318408
f1_score： 0.783653846153846
AUC： 0.7881606765327694




### 資料集2

#### baseline

In [72]:
SVM_model(X_train_2,X_test_2,y_train_2,y_test_2)
LR_model(X_train_2,X_test_2,y_train_2,y_test_2)
RF_model(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model(X_train_2,X_test_2,y_train_2,y_test_2)

SVM baseline
TP: 167
TN: 176
FP: 32
FN: 49
Accuracy： 0.8089622641509434
Recall： 0.7731481481481481
Specificity： 0.8461538461538461
Precision： 0.8391959798994975
f1_score： 0.8048192771084337
AUC： 0.8096509971509971


LR baseline
TP: 161
TN: 175
FP: 33
FN: 55
Accuracy： 0.7924528301886793
Recall： 0.7453703703703703
Specificity： 0.8413461538461539
Precision： 0.8298969072164949
f1_score： 0.7853658536585365
AUC： 0.7933582621082621


RF baseline
TP: 107
TN: 196
FP: 12
FN: 109
Accuracy： 0.714622641509434
Recall： 0.49537037037037035
Specificity： 0.9423076923076923
Precision： 0.8991596638655462
f1_score： 0.6388059701492537
AUC： 0.7188390313390313


AdaBoost_model
TP: 172
TN: 157
FP: 51
FN: 44
Accuracy： 0.7759433962264151
Recall： 0.7962962962962963
Specificity： 0.7548076923076923
Precision： 0.7713004484304933
f1_score： 0.7835990888382688
AUC： 0.7755519943019943




### 資料集3

#### baseline

In [73]:
SVM_model(X_train_3,X_test_3,y_train_3,y_test_3)
LR_model(X_train_3,X_test_3,y_train_3,y_test_3)
RF_model(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model(X_train_3,X_test_3,y_train_3,y_test_3)

SVM baseline
TP: 154
TN: 187
FP: 35
FN: 48
Accuracy： 0.8042452830188679
Recall： 0.7623762376237624
Specificity： 0.8423423423423423
Precision： 0.8148148148148148
f1_score： 0.7877237851662405
AUC： 0.8023592899830524


LR baseline
TP: 149
TN: 191
FP: 31
FN: 53
Accuracy： 0.8018867924528302
Recall： 0.7376237623762376
Specificity： 0.8603603603603603
Precision： 0.8277777777777777
f1_score： 0.7801047120418848
AUC： 0.798992061368299


RF baseline
TP: 126
TN: 196
FP: 26
FN: 76
Accuracy： 0.7594339622641509
Recall： 0.6237623762376238
Specificity： 0.8828828828828829
Precision： 0.8289473684210527
f1_score： 0.7118644067796609
AUC： 0.7533226295602534


AdaBoost_model
TP: 154
TN: 168
FP: 54
FN: 48
Accuracy： 0.7594339622641509
Recall： 0.7623762376237624
Specificity： 0.7567567567567568
Precision： 0.7403846153846154
f1_score： 0.751219512195122
AUC： 0.7595664971902597


